In [78]:
import geocoder
g=geocoder.ip("me")
location = g.latlng
print(location)

[1.2897, 103.8501]


In [80]:
import pandas as pd
import requests
import json
from datetime import datetime

#GSheets Stuff
import gspread
from df2gspread import df2gspread as d2g
from df2gspread import gspread2df as g2d
from oauth2client.service_account import ServiceAccountCredentials

#Credential stuff
scope = ["https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name("./creds.json", scope)
gc = gspread.authorize(credentials)

#Get location data from one set on LTA data
spreadsheet_key = "1ea_-CTzoQZXzrOM6ZW-vdXwYWuaSg5t6mKiZRtmWwX0"
wks_name = 'Sheet1'
df = g2d.download(spreadsheet_key, wks_name, col_names=False, row_names=False, credentials=credentials, start_cell='A1')
df.head()

df[5] = df[5].str.split(" ", n=1, expand=False)
latlng = pd.DataFrame(df[5].to_list(), columns=['lat', 'lng'])
latlng.insert(0, 'ID', df[2])
latlng['lat'] = pd.to_numeric(latlng['lat'], errors='coerce')
latlng['lng'] = pd.to_numeric(latlng['lng'], errors='coerce')

latlng.head()

,ID,lat,lng
0,1,1.29375,103.85718
1,2,1.29115,103.85728
2,4,1.29011,103.85561
3,5,1.29251,103.86009
4,16,1.26421,103.82263


In [84]:
radius = 0.005
near_me = pd.DataFrame(latlng[abs(latlng['lat'] - location[0]) < radius])
near_me = pd.DataFrame(near_me[abs(near_me['lng'] - location[1]) < radius])
near_me

,ID,lat,lng
31,56,1.290270,103.851670
34,59,1.290678,103.846480
41,66,1.291100,103.850200
44,A0011,1.293926,103.848866
55,C0133,1.288266,103.848183
...,...,...,...
240631,66,1.291100,103.850200
240634,A0011,1.293926,103.848866
240645,C0133,1.288266,103.848183
240664,H0024,1.286401,103.847327


In [87]:
near_me.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 734 entries, 31 to 241160
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      734 non-null    object 
 1   lat     734 non-null    float64
 2   lng     734 non-null    float64
dtypes: float64(2), object(1)
memory usage: 22.9+ KB


In [88]:
ID = list(set(near_me["ID"]))
ID

['3', '66', 'HLM', 'H0024', 'A0011', '56', 'C0133', '59']